# Предсказание уровня дохода

Импортируем данные в Pandas Dataframe

In [ ]:
import pandas as pd

In [ ]:
# нам потребуется файл adult.csv
# в этой ячейке пишем код вместе


Основная информация о данных

In [ ]:
data.info()

Краткое описание
http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

Целевая переменная - уровень дохода - более 50K в год или меньше

Данные собраны очень качественно, пропусков нет. 

Для заполнения отлично работает метод fillna

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html

### Готовим данные для модели
Смотрим что можно улучшить

In [ ]:
data['workclass'].unique()

In [ ]:
data[ data['workclass'] == '?' ].head()

Если надо изменить значения определенных строк, то лучше использовать метод loc

In [ ]:
data.loc[ data.workclass == '?', : ].head()

Оставляем только нужный столбец

In [ ]:
data.loc[ data.workclass == '?', 'workclass' ].head()

Назначаем новое значение

In [ ]:
# в этой ячейке пишем код самостоятельно


Проверяем, что изменения произошли

In [ ]:
data['workclass'].unique()

Аналогично для еще двух столбцов

In [ ]:
data.loc[ data['native-country'] == '?', 'native-country' ] = 'undefined'
data['native-country'].unique()

In [ ]:
data.loc[ data['occupation'] == '?', 'occupation' ] = 'undefined'
data['occupation'].unique()

### Посмотрим как влияют значения столбцов на распределение доходов

In [ ]:
data.groupby('workclass').count()

In [ ]:
data_grouped = data.groupby(['workclass', 'income']).count()
data_grouped

Посмотрим распределение для каждого значения столбца 'workclass'

In [ ]:
shares = data_grouped.groupby(level = 0).apply(lambda x: 100 * x / x.sum())['age'].reset_index()
shares

### Пора нарисовать долю '>50K'

In [ ]:
%matplotlib inline

In [ ]:
ax = shares[ shares['income'] == '>50K' ].plot(kind = 'bar')
ax.set_xticklabels(shares.workclass)

### Выберем признаки

In [ ]:
selected_columns = data[ ['age', 'workclass', 'education', 'occupation'] ]
selected_columns.head()

Категориальные признаки не подходят для вычислений. Заменим их на строки из 0 и 1:

In [ ]:
X = pd.get_dummies(selected_columns, columns = ['workclass', 'education', 'occupation'])
X.head()

Целевая переменная - столбец 'income'

In [ ]:
target = data['income']
target.head()

Переведем и эти обозначения в числа в один столбец

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit( target )

In [ ]:
le.classes_

In [ ]:
le.transform( ['<=50K', '>50K', '>50K', '<=50K'] )

In [ ]:
Y = pd.Series( le.transform( data['income'] ) )
Y.head()

### Разбиваем наши данные на обучающую и тестовую выборки

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [ ]:
X_train.head()

In [ ]:
Y_train.head()

### Пора построить модель

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

### Делаем предсказания на тестовой выборке

In [ ]:
predictions = model.predict_proba(X_test)

In [ ]:
for pair in zip( Y, predictions[:, 1] ):
    print(pair)

In [ ]:
model.score(X_test, Y_test)

Можно получить такой результат и вручную:

In [ ]:
i = 0
i_true = 0

for pair in zip( Y_test, model.predict(X_test) ):
    
    if pair[0] == pair[1]:
        i_true += 1
    
    i += 1

In [ ]:
print('Точность предсказаний: {:.2%}'.format(i_true / i))

# L1 и L2 регуляризация

In [ ]:
selected_columns = data[ ['education', 'capital-gain', 'capital-loss'] ]

In [ ]:
# какие столбцы надо прогнать через get_dummies?


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.9, random_state = 0)

In [ ]:
model = LogisticRegression()
model.fit(X_train, Y_train)

In [ ]:
model.score(X_train, Y_train)

In [ ]:
model.score(X_test, Y_test)

Прогоняем несколько значений "силы" регуляризации

In [ ]:
for i, C in enumerate((100, 1, 0.01)):
    print( i, C )

In [ ]:
for i, C in enumerate((100, 1, 0.1, 0.01)):
    
    print('C = {}'.format(C))
    
    model_l1 = LogisticRegression(C=C, penalty='l1')
    model_l2 = LogisticRegression(C=C, penalty='l2')
    
    model_l1.fit(X_train, Y_train)
    model_l2.fit(X_train, Y_train)
    
    print( 'score with L1 penalty: {:.1%} vs {:.1%}'.format(model_l1.score(X_test, Y_test), model_l1.score(X_train, Y_train)) )
    print( 'difference: {:.1%}\n'.format(model_l1.score(X_test, Y_test) - model_l1.score(X_train, Y_train)) )
    
    print( 'score with L2 penalty: {:.1%} vs {:.1%}'.format(model_l2.score(X_test, Y_test), model_l2.score(X_train, Y_train)) )
    print( 'difference: {:.1%}\n'.format(model_l2.score(X_test, Y_test) - model_l2.score(X_train, Y_train)) )

# Немного практики

Попробуйте улучшить точность модели

In [ ]:
selected_columns = data[ ['', ''] ]

In [ ]:
X = pd.get_dummies(selected_columns, columns = [''])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.9, random_state = 0)

In [ ]:
model = LogisticRegression()
model.fit(X_train, Y_train)

In [ ]:
model.score(X_test, Y_test)